# Example to get energies (total, electronic, nuclear, heat of formation, orbital), charges, dipole, forces, density matrix.

In [1]:
import sys
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
#device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%time

### create molecule object:
species = torch.as_tensor([[8,6,1,1],
                           [8,6,1,1],
                           [8,8,6,0]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.23,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [0.0,0.0,0.0]            # zero-padding for batching
                              ]
                            ], device=device)

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

seqm_parameters = {
                   'method' : 'PM6_SP',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True # store orbital energies
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules)

CPU times: user 1.87 s, sys: 893 ms, total: 2.77 s
Wall time: 2.84 s


In [3]:
molecules.nocc

tensor([6, 6, 8], device='cuda:0')

In [4]:
print(' Total Energy (eV):\n', molecules.Etot)
print('\n Electronic Energy (eV): ', molecules.Eelec)
print('\n Nuclear Energy (eV):\n', molecules.Enuc)
print('\n Heat of Formation (ev):\n', molecules.Hf)
print('\n HOMO-LUMO gap (eV):\n', molecules.e_gap)
print('\n Orbital energies (eV):\n', molecules.e_mo)
print('\n Charges:\n', molecules.q)
print('\n Dipole:\n', molecules.d)
print('\n Forces (eV/A):\n', molecules.force)

 Total Energy (eV):
 tensor([-440.2169, -440.2169, -696.4774], device='cuda:0')

 Electronic Energy (eV):  tensor([ -822.5876,  -822.5876, -1379.9633], device='cuda:0')

 Nuclear Energy (eV):
 tensor([382.3707, 382.3707, 683.4859], device='cuda:0')

 Heat of Formation (ev):
 tensor([-0.8825, -0.8825,  5.5543], device='cuda:0')

 HOMO-LUMO gap (eV):
 tensor([10.1862, 10.1862,  7.9755], device='cuda:0')

 Orbital energies (eV):
 tensor([[-3.1726e+01, -2.3382e+01, -1.6982e+01, -1.5154e+01, -1.4632e+01,
         -1.0175e+01,  1.0996e-02,  3.9540e+00,  4.4427e+00,  4.6379e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-3.1726e+01, -2.3382e+01, -1.6982e+01, -1.5154e+01, -1.4632e+01,
         -1.0175e+01,  1.0996e-02,  3.9540e+00,  4.4427e+00,  4.6379e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-3.7972e+01, -2.9900e+01, -2.2974e+01, -1.9398e+01, -1.8979e+01,
     

In [5]:
print('\n Density matrix:\n', molecules.dm)


 Density matrix:
 tensor([[[ 1.8598e+00, -3.0431e-01,  0.0000e+00,  0.0000e+00,  1.5452e-01,
          -3.7303e-01,  0.0000e+00,  0.0000e+00,  5.0731e-02,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  5.0731e-02,  0.0000e+00,  0.0000e+00,
           0.0000e+00],
         [-3.0431e-01,  1.2811e+00,  0.0000e+00,  0.0000e+00,  5.3410e-01,
          -7.2955e-01,  0.0000e+00,  0.0000e+00, -7.3661e-02,  0.0000e+00,
           0.0000e+00,  0.0000e+00, -7.3661e-02,  0.0000e+00,  0.0000e+00,
           0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.9345e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  2.4753e-01,  0.0000e+00, -1.8099e-01,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  1.8099e-01,  0.0000e+00,  0.0000e+00,
           0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.3462e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  9.3818e-01,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         

In [6]:
molecules.coordinates

tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 1.2200,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 1.8200, -0.9400,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 1.2200,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 1.8200, -0.9400,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 1.2300,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]], device='cuda:0', requires_grad=True)